In [18]:
!pip install pyTelegramBotAPI
!pip install transformers
!pip install python-telegram-bot
!pip install retry

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 9.2 MB/s eta 0:00:00


In [19]:
import logging
import requests
import telebot
from retry import retry

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

# Set up Telegram Bot API token
bot_token = 'YOUR_BOT_TOKEN'
bot = telebot.TeleBot(bot_token)

# Define a command handler for /start
@bot.message_handler(commands=['start'])
def start(message):
    bot.reply_to(message, "Welcome! Please send me a text and I'll send you an image.")

# Define a message handler
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    # Get the input text from the user
    input_text = message.text
    
    # Generate an image based on the input text (Replace this with your own logic to generate the image)
    image_url = generate_image_url(input_text)
    
    # Send the image to the user
    if image_url:
        bot.send_photo(chat_id=message.chat.id, photo=image_url)
    else:
        bot.reply_to(message, "Sorry, I couldn't find an image for that text.")

# Retry decorator with exponential backoff
@retry(tries=3, delay=2, backoff=2)
def make_api_request(url, params):
    response = requests.get(url, params=params, timeout=10)
    response.raise_for_status()
    return response

# Function to generate an image URL based on the input text
def generate_image_url(input_text):
    # Replace this with your own logic to generate the image URL
    url = "https://api.unsplash.com/search/photos"
    params = {
        "query": input_text,
        "client_id": "YOUR_UNSPLASH_API_ACCESS_KEY"
    }
    try:
        response = make_api_request(url, params)
        data = response.json()
        if data.get("results"):
            image_url = data["results"][0]["urls"]["regular"]
            return image_url
    except Exception as e:
        logging.error("Error generating image URL: %s", str(e))
    return None

# Start the bot
bot.polling()


ApiTelegramException: ignored